In [58]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
import json
import os

In [2]:
transaction_detail_df = pd.read_csv("../transaction_detail_first_batch.csv", low_memory=False)

In [5]:
for col in transaction_detail_df.columns:
    print(col)

Address
Address Book Type
Agent
Aggregate
Amount
Associations
Check
CoSigner Obligations
Description
Due Date
Employer Name
Exam Letter Date
Filed Date
In-Kind/Independent Expenditures
Interest Rate
Name
Occupation
Occupation Letter Date
Payer of Personal Expenditure
Payment Method
Process Status
Purpose
Repayment Schedule
Transaction Date
Transaction ID
Transaction Sub Type
Transaction Type


In [18]:
business_sub_df = transaction_detail_df[transaction_detail_df["Address Book Type"] == "Business Entity"].copy()

In [21]:
len(np.unique(list(business_sub_df["Name"])))/len(business_sub_df)

0.2108880007798031

Create a dict of arrays: original_name: [[% match, name], ...]

In [24]:
np.unique(list(business_sub_df["Name"]))[10]

'2030 Investors LLC'

In [26]:
query = np.unique(list(business_sub_df["Name"]))[10]


In [29]:
# Get a list of matches ordered by score, default limit to 5
choices = np.unique(list(business_sub_df["Name"]))

[('2030 Investors LLC', 100),
 ('3701 Investors LLC', 89),
 ('111th Square, LLC - Sanchez Family Trust', 86),
 ('525, LLC', 86),
 ('6 Foot 8 LLC', 86),
 ('76 Words LLC', 86),
 ('ADG 3 LLC', 86),
 ('ADP LLC', 86),
 ('ADP, LLC', 86),
 ('ART LLC', 86),
 ('Active Family Chiropractic LLC', 86),
 ('Advantage Heating & Air Conditioning LLC', 86),
 ('Advantage Professional Management, LLC', 86),
 ('AllCare Health Management Services, LLC', 86),
 ('AllCare Management Services, LLC', 86),
 ('Andaluz LLC', 86),
 ('Appropriate Development Group III, LLC', 86),
 ('Arms LLC', 86),
 ('BEST hq LLC', 86),
 ('BTU LLC', 86)]

In [38]:
a = list(range(0,10))
for i,n in enumerate(a):
    print(n)
    print(a[:i] + a[i+1:])

0
[1, 2, 3, 4, 5, 6, 7, 8, 9]
1
[0, 2, 3, 4, 5, 6, 7, 8, 9]
2
[0, 1, 3, 4, 5, 6, 7, 8, 9]
3
[0, 1, 2, 4, 5, 6, 7, 8, 9]
4
[0, 1, 2, 3, 5, 6, 7, 8, 9]
5
[0, 1, 2, 3, 4, 6, 7, 8, 9]
6
[0, 1, 2, 3, 4, 5, 7, 8, 9]
7
[0, 1, 2, 3, 4, 5, 6, 8, 9]
8
[0, 1, 2, 3, 4, 5, 6, 7, 9]
9
[0, 1, 2, 3, 4, 5, 6, 7, 8]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [62]:
top_choices_dict = {}
start_time = time.time()
full_list = list(np.unique(list(business_sub_df["Name"])))
for i,name in enumerate(full_list):
    query = name
    choices = full_list[:i] + full_list[i+1:]
    top_choices_dict[name] = process.extract(query, choices, limit=20)
    if i % 100 == 0:
        print(str(np.round(i/len(full_list),3) * 100) + "%")
        print(str(time.time() - start_time))
        with open("match_dict.json", 'a') as f:
            json.dump(top_choices_dict, f, indent=4)
        print(os.stat("match_dict.json").st_size, " ", len(top_choices_dict))
        del(top_choices_dict)
        top_choices_dict = {}
print(str(np.round(i/len(full_list),3) * 100) + "%")
print(str(time.time() - start_time))
with open("match_dict.json", 'a') as f:
    json.dump(top_choices_dict, f, indent=4)

0.0%
0.46144986152648926
1661   1
2.3%
47.58176398277283
151385   100
4.6%
97.72590899467468
302936   100
6.9%
145.89507913589478
453226   100
9.2%
192.75103211402893
603535   100
11.600000000000001%
239.15482211112976
754360   100
13.900000000000002%
285.6901149749756
904356   100
16.2%
331.0252468585968
1053517   100
18.5%
377.261990070343
1207616   100
20.8%
424.3091068267822
1359077   100
23.1%
468.8106610774994
1507819   100
25.4%
513.2184240818024
1658468   100
27.700000000000003%
559.4051880836487
1808444   100
30.0%
607.9299409389496
1958870   100
32.4%
654.3247210979462
2107796   100
34.699999999999996%
700.55127120018
2256289   100
37.0%
748.3388550281525
2404971   100
39.300000000000004%
795.2689678668976
2553698   100
41.6%
841.7601451873779
2702503   100
43.9%
887.5808119773865
2852549   100
46.2%
933.0598320960999
2999533   100
48.5%
981.3507170677185
3150148   100
50.8%
1027.3287360668182
3298216   100
53.2%
1073.274889945984
3450776   100
55.50000000000001%
1119.0151929